In [2]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from scipy import stats,sparse
from sklearn.base import TransformerMixin
from datetime import datetime as dt
from math import isnan
from numpy import ma
import pickle as pickle
import xgboost as xgb
import time
from pandas import *
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier 

import json
from sklearn.metrics import roc_curve, auc
from re import sub
from collections import defaultdict
import scipy as sp

from sklearn.cross_validation import StratifiedKFold,cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report,confusion_matrix
from sklearn import linear_model
from sklearn import metrics

from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from scipy import sparse
from sklearn.preprocessing import OneHotEncoder,StandardScaler


In [ ]:
FeaturesSelected=np.load('./mostrecentinput/SDGLogistic_Oct172015032138_AUC_0.768923541152.p')
X=np.load('./mostrecentinput/nxtrain_standard_original0.npy')
X1= np.load('./mostrecentinput/nxtrain_standard_derived0.npy')
X2=pd.read_pickle("./mostrecentinput/time_series_derived_standard_train2.dat")
X3=pd.read_pickle("./mostrecentinput/time_series_original_standard_train2.dat")
X=np.hstack((X,X1,X2,X3))
del X1,X2,X3
X1=X*1000
X1=X1.astype(int)
X1=np.delete(X1, FeaturesSelected,axis=1)
X1.dump('NumericalVariable_ConvertedCatVariables.p')

In [ ]:
X_test=np.load('./mostrecentinput/nxtest_standard_original0.npy')
X_test1= np.load('./mostrecentinput/nxtest_standard_derived0.npy')
X_test2=pd.read_pickle("./mostrecentinput/time_series_derived_standard_test2.dat")
X_test3=pd.read_pickle("./mostrecentinput/time_series_original_standard_test2.dat")
X_test=np.hstack((X_test,X_test1,X_test2,X_test3))
xtest0=X_test[:,FeaturesSelected]
del X_test1,X_test2,X_test3
X_test=X_test*1000
X_test=X_test.astype(int)
X_test1=np.delete(X_test, FeaturesSelected,axis=1)
X_test1.dump('Test_NumericalVariable_ConvertedCatVariables.p')

In [ ]:
train=np.load('Train_NumericalVariable_ConvertedCatVariables.p')
test=np.load('Test_NumericalVariable_ConvertedCatVariables.p')
y=pickle.load(open("./mostrecentinput/ytrain2.dat","rb"))

In [ ]:
traintest1=np.vstack((train,test))
traintest1=traintest1.apply(lambda x: x - x.min() if x.min() <= 0 else x)
hot = OneHotEncoder()
data_mt = hot.fit_transform(traintest1)
data_mt_train=data_mt[:145231,:]
data_mt_test=data_mt[145231:,:]
del data_mt

Kfeature=round(data_mt_train.shape[1]*0.2)
SLTKBest=SelectKBest(chi2, k=Kfeature)
SLTKBest.fit(data_mt_train, y)
SparseSelect=SLTKBest.get_support()
Xtrain_sparse40Percent_Seg1 = data_mt_train[:,SparseSelect]
    